<a href="https://colab.research.google.com/github/GuilhermeJobs/Endere-os/blob/main/Shech_lat_log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm.auto import tqdm  # Importar tqdm para barras de progresso automáticas (funciona bem no Colab)
import time  # Pode ser útil para adicionar pausas extras se necessário

In [ ]:
url_raw = 'https://raw.githubusercontent.com/GuilhermeJobs/Endere-os/main/Alunos_enderecos.xlsx'
try:
    df = pd.read_excel(url_raw, engine='openpyxl')
    print("DataFrame carregado com sucesso.")
    print(df.head())
    print(f"\nNúmero de linhas no DataFrame: {len(df)}")
except Exception as e:
    print(f"Erro ao carregar o DataFrame: {e}")
    # Interromper a execução se o DF não puder ser carregado
    raise SystemExit("Não foi possível carregar os dados. Verifique a URL e a conexão.")

DataFrame carregado com sucesso.
     Id                                       Rua, Bairro
0  3907  RUA SEBASTIÃO FERREIRA, RESIDENCIAL ARACY AMARAL
1  3467       RUA CELINA AMARAL, RESIDENCIAL ARACY AMARAL
2  7485          RUA VICINAL 5, CHÁCARAS VARGEM BONITA II
3  6467   RUA TRAJANO BONIFÁCIO, RESIDENCIAL ARACY AMARAL
4  6678          RUA VICINAL 4, CHÁCARAS VARGEM BONITA II

Número de linhas no DataFrame: 715


In [ ]:
locator = Nominatim(user_agent="MeuAplicativoDeGeocodificacaoColab_v1")

In [ ]:
def get_location(address):
    """Tenta geocodificar um endereço e retorna (latitude, longitude) ou (None, None)."""
    if pd.isna(address) or not isinstance(address, str) or address.strip() == "":
        return None, None  # Retorna None para endereços vazios ou inválidos
    try:
        location = locator.geocode(address, timeout=10) # Timeout de 10 segundos
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Erro ao geocodificar '{address}': {e}")
        return None, None # Retorna None em caso de erro na requisição

In [ ]:
geocode_with_delay = RateLimiter(get_location, min_delay_seconds=1, return_value_on_exception=(None, None))

In [ ]:
tqdm.pandas(desc="Geocodificando Endereços")

In [ ]:
print("\nIniciando geocodificação (pode levar tempo dependendo do número de endereços)...")
df['Coordinates'] = df['Rua, Bairro'].progress_apply(geocode_with_delay)


Iniciando geocodificação (pode levar tempo dependendo do número de endereços)...


Geocodificando Endereços:   0%|          | 0/715 [00:00<?, ?it/s]

In [ ]:
df[['latitude', 'longitude']] = pd.DataFrame(df['Coordinates'].tolist(), index=df.index)

In [ ]:
df = df.drop(columns=['Coordinates'])
print("\nGeocodificação concluída!")

In [ ]:
num_na = df['latitude'].isna().sum()
print(f"\nNúmero de endereços não encontrados (NaN): {num_na} de {len(df)}")


Número de endereços não encontrados (NaN): 136 de 715


In [ ]:
df

,Id,"Rua, Bairro",latitude,longitude
0,3907,"RUA SEBASTIÃO FERREIRA, RESIDENCIAL ARACY AMARAL",-16.739350,-49.073655
1,3467,"RUA CELINA AMARAL, RESIDENCIAL ARACY AMARAL",-16.739188,-49.071172
2,7485,"RUA VICINAL 5, CHÁCARAS VARGEM BONITA II",-16.758662,-49.087988
3,6467,"RUA TRAJANO BONIFÁCIO, RESIDENCIAL ARACY AMARAL",-16.739814,-49.070672
4,6678,"RUA VICINAL 4, CHÁCARAS VARGEM BONITA II",-16.758662,-49.087988
...,...,...,...,...
710,4822,"RUA 3, SOLAR PARK ILHABELA",-16.737532,-49.080502
711,8125,"RUA RR-15, RESIDENCIAL RAVENA",-22.660229,-50.411428
712,7264,"RUA CECI BLOCOS, SETOR MONTE AZUL",-16.733675,-49.087622
713,5104,"RUA ORLANDO FERREIRA, JARDIM DO LAGO",-16.676714,-49.099815
